# Introduction
Named Entity Recognition (NER) is the process of identifying named entities in text. Example of named entities are: "Person", "Location", "Organization", "Dates" etc. NER is essentially a token classification task where every token is classified into one or more predetermined categories.

In this exercise, we will train a simple Transformer based model to perform NER. We will be using the data from CoNLL 2003 shared task.

In [1]:
!pip3 install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.8 MB/s 
     |████████████████████████████████| 212 kB 63.4 MB/s 
     |████████████████████████████████| 115 kB 65.4 MB/s 
     |████████████████████████████████| 120 kB 70.3 MB/s 
     |████████████████████████████████| 127 kB 56.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datasets import load_dataset
from collections import Counter

### Create Transformer Layer

#### First part - TransformerBlock

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):

        # Initialize super class - Keras Layer
        super(TransformerBlock, self).__init__()
        # Initialize MHA Layer
        self.att = keras.layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        # Initialize Feed Forward network
        self.ffn = keras.Sequential(
            [
                keras.layers.Dense(ff_dim, activation="relu"),
                keras.layers.Dense(embed_dim),
            ]
        )
        # Initialize LayerNorm layers
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        # Initialize dropout layers
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training=False):
        # Order the layers as per transformer architecture
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

#### Second Part - Token and Positional Embedding

In [4]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        # Initialize super class - Keras layer
        super(TokenAndPositionEmbedding, self).__init__()
        # Initialize Token Embedding layer
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # Intialize Positional Embedding layer
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, inputs):
        # Add Positional embedding to token embeddings
        maxlen = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        position_embeddings = self.pos_emb(positions)
        token_embeddings = self.token_emb(inputs)
        return token_embeddings + position_embeddings

#### Build the NER model class as a keras.Model subclass

In [5]:
class NERModel(keras.Model):
    def __init__(
        self, num_tags, vocab_size, maxlen=128, embed_dim=32, num_heads=2, ff_dim=32
    ):
        # Initialize Super Class - Keras Model
        super(NERModel, self).__init__()
        # Create embedding - sum of word embedding and positional embedding
        self.embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
        # Create Transformer block
        self.transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
        # Add dropout layer
        self.dropout1 = layers.Dropout(0.1)
        # Add feed forward layer
        self.ff = layers.Dense(ff_dim, activation="relu")
        # Add another dropout layer
        self.dropout2 = layers.Dropout(0.1)
        # Final Feed forward layer - number of neurons matches the number of tags
        self.ff_final = layers.Dense(num_tags, activation="softmax")

    def call(self, inputs, training=False):
        x = self.embedding_layer(inputs)
        x = self.transformer_block(x)
        x = self.dropout1(x, training=training)
        x = self.ff(x)
        x = self.dropout2(x, training=training)
        x = self.ff_final(x)
        return x

### Load the CoNLL 2003 dataset from the datasets library and process it

In [6]:
conll_data = load_dataset("conll2003")

Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/9a4d16a94f8674ba3466315300359b0acd891b68b6c8743ddf60b9c702adce98. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [29]:
def prepare_arrow_data(data):
  output = []
  for record in data:
    ner_tags = record["ner_tags"]
    tokens = record["tokens"]
    if len(tokens) > 0:
      output.append(
          str(len(tokens))
          + "\t"
          + "\t".join(tokens)
          + "\t"
          + "\t".join(map(str, ner_tags))
      )

  return output

In [ ]:
batch_size = 32

train_slices = prepare_arrow_data(conll_data['train'])
val_slices = prepare_arrow_data(conll_data['validation'])

train_data = tf.data.Dataset.from_tensor_slices(train_slices).batch(batch_size)
val_data = tf.data.Dataset.from_tensor_slices(val_slices).batch(batch_size)